In [2]:
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats.mstats import winsorize
import re

In [3]:
df = pd.read_csv('../data/full_survey_data.csv')

/var/folders/b3/glj_22g92pl25nkdvjvxnzz00000gn/T/ipykernel_37360/4096187165.py:1: DtypeWarning: Columns (10,11,12,15,16,17,18,19,20,21,22,24,27,37,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,69,74,75,76,78,88,91,94,95,104,113,114,115,116,118,121,122,123,124,125,132,135,137,138,141,142,151,160,161,162,163,165,168,169,170,171,172,179,182,184,185,188,189,198,207,208,209,210,212,215,216,217,218,219,226,229,231,232,236,245,254,255,256,257,259,262,263,264,265,266,273,276,278,279,283,292,301,302,303,304,306,309,310,312,313,320,325,326,330,339,348,349,350,351,353,356,357,359,360,367,372,373,377,386,395,396,397,398,400,403,404,406,407,414,419,420,424,433,443,444,445,446,447,448,450,451,452,453,454,455,456,458,459,460,461,462,463,464,465,466,467,469,470,471,472,473,474,475,476,478,479,480,481,482,500,501,502,508,510,515,516,522,524,525,526,533,538,547,556,561,565,566,567,574,580,581,582,591,600,603,607,608,609,616,620,624,633,680,681,682,687,688,694,696,697,698,705,71

In [4]:
#  x ~ z
# creating the z variable and we only consider animals that potentially use crop residue as feed
# Step 1: Select relevant columns
name_cols = [f'livestock_name_{i}' for i in range(1, 6)]
died_cols = [f'livestock_died_{i}' for i in range(1, 6)]

# Step 2: Melt the livestock names and deaths
names_long = df.melt(id_vars='id_unique', value_vars=name_cols, 
                     var_name='animal_slot', value_name='livestock_name')

died_long = df.melt(id_vars='id_unique', value_vars=died_cols, 
                    var_name='animal_slot', value_name='livestock_died')

# Step 3: Make sure they align by extracting index number
names_long['slot'] = names_long['animal_slot'].str.extract(r'(\d+)').astype(int)
died_long['slot'] = died_long['animal_slot'].str.extract(r'(\d+)').astype(int)

# Step 4: Merge the two long DataFrames
long_df = pd.merge(
    names_long[['id_unique', 'slot', 'livestock_name']],
    died_long[['id_unique', 'slot', 'livestock_died']],
    on=['id_unique', 'slot']
)

# Optional: drop rows where animal name is missing
long_df = long_df.dropna(subset=['livestock_name']).reset_index(drop=True)

# Result: Each row is (id_unique, livestock_name, livestock_died)
print(long_df.head())

death_df = long_df[long_df.livestock_died.notna()]

         id_unique  slot livestock_name  livestock_died
0  bf_adn_2019_1_1     1          sheep             0.0
1  bf_adn_2019_2_1     1         cattle             0.0
2  bf_adn_2019_3_1     1           pigs             3.0
3  bf_adn_2019_4_1     1         cattle             0.0
4  bf_adn_2019_5_1     1         cattle             0.0


In [5]:
# List of animals to keep
animals_to_keep = ['cattle', 'chicken', 'goats', 'sheep', 'donkeys_horses', 'buffalo', 'pigs']

# Filter the DataFrame
final_death_df = death_df[death_df['livestock_name'].isin(animals_to_keep)]
final_death_df

,id_unique,slot,livestock_name,livestock_died
0,bf_adn_2019_1_1,1,sheep,0.0
1,bf_adn_2019_2_1,1,cattle,0.0
2,bf_adn_2019_3_1,1,pigs,3.0
3,bf_adn_2019_4_1,1,cattle,0.0
4,bf_adn_2019_5_1,1,cattle,0.0
...,...,...,...,...
81610,tz_glv_2017_156_1,5,donkeys_horses,0.0
81614,tz_glv_2017_522_1,5,pigs,0.0
81615,tz_glv_2017_562_1,5,donkeys_horses,0.0
81616,tz_glv_2017_889_1,5,chicken,3.0


In [6]:
# Sum livestock deaths per farm
deaths_by_farm = (
    final_death_df.groupby("id_unique")["livestock_died"]
    .sum()
    .reset_index(name="total_livestock_died")
)

deaths_by_farm.head()

,id_unique,total_livestock_died
0,bf_adn_2019_10_1,3.0
1,bf_adn_2019_11_1,5.0
2,bf_adn_2019_12_1,1.0
3,bf_adn_2019_13_1,3.0
4,bf_adn_2019_14_1,7.0


In [7]:


# Extract the data column
values = deaths_by_farm["total_livestock_died"].values

# Winsorize only the top 5%
winsorized_values = winsorize(values, limits=(0, 0.05))
# Create a new column with the winsorized values
deaths_by_farm["winsorized_deaths"] = winsorized_values

deaths_by_farm

,id_unique,total_livestock_died,winsorized_deaths
0,bf_adn_2019_10_1,3.0,3.0
1,bf_adn_2019_11_1,5.0,5.0
2,bf_adn_2019_12_1,1.0,1.0
3,bf_adn_2019_13_1,3.0,3.0
4,bf_adn_2019_14_1,7.0,7.0
...,...,...,...
33063,zm_scn_2017_95_1,0.0,0.0
33064,zm_scn_2017_96_1,1.0,1.0
33065,zm_scn_2017_98_1,0.0,0.0
33066,zm_scn_2017_99_1,1.0,1.0


In [8]:


# 1. Keep only columns that do NOT end with "_<digit(s)>"
cols_to_keep = [c for c in df.columns if not re.search(r"_\d+$", c)]
df_filtered = df[cols_to_keep]

# 2. Merge with deaths_by_farm on id_unique
merged_df = df_filtered.merge(deaths_by_farm, on="id_unique", how="right")

In [9]:
merged_df

,id_unique,id_hh,id_rhomis_dataset,id_form,id_proj,year,gps_lat_rounded,gps_lon_rounded,gps_alt,gps_source,...,quality_reliability,end_time_user,endtime_auto,endtime_calculated,x_submission_time,x_tags,x_version,x_duration,total_livestock_died,winsorized_deaths
0,bf_adn_2019_10_1,51e059049386839a3fbd0fef4c0b99ea,965a6c0a72bf4bf0079b305ee61661e4,bf_adn_2019,adn,2019,11.2,-1.0,302.000000,survey,...,4.0,03/04/2019 07:32,03/04/2019 07:35,08:35:04,08/04/2019 10:02,NaN,2.0,2187.0,3.0,3.0
1,bf_adn_2019_11_1,3823ee57a507ffd5712f694e7d0b2aa6,965a6c0a72bf4bf0079b305ee61661e4,bf_adn_2019,adn,2019,11.2,-1.0,334.000000,survey,...,4.0,03/04/2019 08:18,03/04/2019 08:18,09:18:27,08/04/2019 10:02,NaN,2.0,2418.0,5.0,5.0
2,bf_adn_2019_12_1,06b16d26bb4c89444f41bd4cbe43788b,965a6c0a72bf4bf0079b305ee61661e4,bf_adn_2019,adn,2019,11.2,-1.0,350.000000,survey,...,4.0,03/04/2019 09:37,03/04/2019 09:39,10:39:08,08/04/2019 10:02,NaN,2.0,2592.0,1.0,1.0
3,bf_adn_2019_13_1,30e4bca395c0c6651b211a7e6134d6f9,965a6c0a72bf4bf0079b305ee61661e4,bf_adn_2019,adn,2019,11.2,-1.0,332.000000,survey,...,4.0,03/04/2019 10:11,03/04/2019 10:13,11:13:14,08/04/2019 10:02,NaN,2.0,2030.0,3.0,3.0
4,bf_adn_2019_14_1,e3517548f76d6af4b55a35fa3871bc89,965a6c0a72bf4bf0079b305ee61661e4,bf_adn_2019,adn,2019,11.2,-1.0,314.000000,survey,...,4.0,03/04/2019 11:10,03/04/2019 11:58,12:58:43,08/04/2019 10:02,NaN,2.0,6187.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33063,zm_scn_2017_95_1,76695ab44b30317a43d3cc24e60b5cf4,d94bd7e89f59c72c3ee9d0a0a8337abc,zm_scn_2017,scn,2017,-14.6,31.2,1081.386059,survey,...,5.0,16:26:00.000+02,2017-05-29t16:26:09.050+02,16:26:09,2017-05-30t05:18:03,NaN,1.0,NaN,0.0,0.0
33064,zm_scn_2017_96_1,b9473901f9ca0ca74650f6bb36009d6c,d94bd7e89f59c72c3ee9d0a0a8337abc,zm_scn_2017,scn,2017,-14.4,31.3,912.000000,survey,...,3.0,10:27:00.000+02,2017-05-30t10:33:17.862+02,10:33:17,2017-05-30t13:15:25,NaN,1.0,NaN,1.0,1.0
33065,zm_scn_2017_98_1,82cd2e85f8c5da731379f1f928786fa0,d94bd7e89f59c72c3ee9d0a0a8337abc,zm_scn_2017,scn,2017,-14.4,31.3,933.000000,survey,...,4.0,12:02:00.000+02,2017-05-30t12:02:42.079+02,12:02:42,2017-05-30t13:18:35,NaN,1.0,NaN,0.0,0.0
33066,zm_scn_2017_99_1,c92f9ade8c80e0412ade0c26ba02f770,d94bd7e89f59c72c3ee9d0a0a8337abc,zm_scn_2017,scn,2017,-14.4,31.3,955.000000,survey,...,3.0,12:49:00.000+02,2017-05-30t12:49:30.545+02,12:49:30,2017-05-30t13:19:20,NaN,1.0,NaN,1.0,1.0


In [10]:
num_df = merged_df.select_dtypes(include="number")

# 2. Correlations with winsorized_deaths
corr_winsor = num_df.corrwith(num_df["winsorized_deaths"]).dropna()

# Drop self-correlation and sort
corr_winsor = corr_winsor.drop("winsorized_deaths").sort_values(ascending=False)

# 3. Correlations with total_livestock_died
corr_total = num_df.corrwith(num_df["total_livestock_died"]).dropna()

# Drop self-correlation and sort
corr_total = corr_total.drop("total_livestock_died").sort_values(ascending=False)

# 4. Show top 20 for each
print("\nTop 20 correlations with winsorized_deaths:")
print(corr_winsor.head(20))

print("\nTop 20 correlations with total_livestock_died:")
print(corr_total.head(20))


Top 20 correlations with winsorized_deaths:
wool_sold_income                  0.501103
livestock_count                   0.278271
livestock_heads_dogs              0.270825
livestock_heads_goats             0.260037
cheese_amount                     0.253297
livestock_heads_guinea_pigs       0.252536
livestock_heads_duck              0.241625
livestock_heads_cattle            0.213473
livestock_heads_sheep             0.174347
livestock_heads_otherpoultry      0.137406
livestock_heads_chicken           0.136291
crop_product_sold_income          0.133474
livestock_heads_donkeys_horses    0.124688
butter_sold_income                0.107632
crop_count                        0.091909
age_femalehead                    0.091339
livestock_heads_oxen              0.076156
age_malehead                      0.075255
malesover50                       0.064272
livestock_heads_pigs              0.063951
dtype: float64

Top 20 correlations with total_livestock_died:
wool_sold_income                

/Users/nishantshrestha/Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/nishantshrestha/Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/nishantshrestha/Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/nishantshrestha/Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [11]:
print(corr_winsor)


wool_sold_income                  0.501103
livestock_count                   0.278271
livestock_heads_dogs              0.270825
livestock_heads_goats             0.260037
cheese_amount                     0.253297
livestock_heads_guinea_pigs       0.252536
livestock_heads_duck              0.241625
livestock_heads_cattle            0.213473
livestock_heads_sheep             0.174347
livestock_heads_otherpoultry      0.137406
livestock_heads_chicken           0.136291
crop_product_sold_income          0.133474
livestock_heads_donkeys_horses    0.124688
butter_sold_income                0.107632
crop_count                        0.091909
age_femalehead                    0.091339
livestock_heads_oxen              0.076156
age_malehead                      0.075255
malesover50                       0.064272
livestock_heads_pigs              0.063951
femalesover50                     0.060782
females25to50                     0.041513
cheese_sold_income                0.040569
gps_lon_rou

In [16]:
print(corr_total)

wool_sold_income                  0.438247
livestock_heads_guinea_pigs       0.239645
livestock_heads_duck              0.234006
cheese_amount                     0.230015
livestock_heads_otherpoultry      0.130720
livestock_heads_camel             0.120380
crop_product_sold_income          0.109308
butter_sold_income                0.107632
other_dairy_amount                0.067678
livestock_heads_chicken           0.067584
livestock_heads_dogs              0.054184
livestock_heads_rabbits           0.041078
cheese_sold_income                0.028319
winsorized_deaths                 0.018911
butter_amount                     0.018346
livestock_heads_goats             0.009447
livestock_heads_buffalo           0.007990
quality_reliability               0.006198
offfarm_incomes_count             0.004857
year                              0.004661
females11to24                     0.004623
gps_alt                           0.004615
gps_lon_rounded                   0.004542
males11to24

In [18]:
df.country.unique()

array(['burkina_faso', 'burundi', 'bolivia', 'drc', "cote d'ivoire",
       'costa_rica', 'ecuador', 'ethiopia', 'ghana', 'gambia',
       'guatemala', 'honduras', 'india', 'kenya', 'cambodia', 'comoros',
       'morocco', 'mali', 'Mali', 'malawi', 'niger', 'nigeria',
       'nicaragua', 'nepal', 'peru', 'philipines', 'palestine', 'rwanda',
       'sierra leone', 'senegal', 'el_salvador', 'tanzania', 'uganda',
       'vietnam', 'south africa', 'zambia'], dtype=object)